Fire detection using YOLOv3 darknet
---

In [ ]:
!apt update
!apt upgrade -y
!uname -m && cat /etc/*release 
!gcc --version
!uname -r 

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downlo

In [ ]:
ls

sample_data/


To access Google Drive Folder and Files

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls -a "/content/drive/MyDrive/MPVI/Zavrsni/fire_detection"

backup		    obj        obj.zip	    YOLOv3.ipynb
generate_test.py    obj.data   results	    yolov4-obj.cfg
generate_train.py   obj.names  test.zip     yolov4-obj-for-testing.cfg
.ipynb_checkpoints  obj_old    WORKING_DIR


**Original Version of Darknet**

In [ ]:
!apt install libopencv-dev python-opencv ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
libopencv-dev is already the newest version (3.2.0+dfsg-4ubuntu0.1).
The following packages were automatically installed and are no longer required:
  linux-headers-4.15.0-143 linux-headers-4.15.0-143-generic
Use 'apt autoremove' to remove them.
The following NEW packages will be installed:
  python-opencv
0 upgraded, 1 newly installed, 0 to remove and 4 not upgraded.
Need to get 535 kB of archives.
After this operation, 2,944 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 python-opencv amd64 3.2.0+dfsg-4ubuntu0.1 [535 kB]
Fetched 535 kB in 1s (496 kB/s)
Selecting previously unselected package python-opencv.
(Reading database ... 190093 files and directories currently installed.)
Preparing to unpack .../python-opencv_3.2.0+dfsg-4ubuntu0.1_amd64.deb ...
Unpacking python-o

Changing the variables to include OpenCV and GPU in the Makefile

In [ ]:
!sed -i 's/OPENCV=0/OPENCV=1/g' Makefile
!sed -i 's/GPU=0/GPU=1/g' Makefile
%pycat Makefile

In [ ]:
!make

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1139:10: note: ...this statement

**How to train (to detect your custom objects):**
---

**Training Yolo v3:**

1.Create file yolo-obj.cfg with the same content as in yolov3.cfg (or copy yolov3.cfg to yolo-obj.cfg) and:

* change line batch to batch=64
* change line subdivisions to subdivisions=8
* change line classes=80 to your number of objects in each of 3 [yolo]-layers:
  * yolov3.cfg#L610
  * yolov3.cfg#L696
  * yolov3.cfg#L783
* change [filters=255] to filters=(classes + 5)x3 in the 3 [convolutional] before each [yolo] layer
  * yolov3.cfg#L603
  * yolov3.cfg#L689
  * yolov3.cfg#L776
  
So if classes=1 then should be filters=18. If classes=2 then write filters=21.

In [ ]:
%cp cfg/yolov3.cfg cfg/yolo-obj.cfg
!sed -i 's/batch=1/batch=64/g' cfg/yolo-obj.cfg
!sed -i 's/subdivisions=1/subdivisions=32/g' cfg/yolo-obj.cfg
!sed -i 's/classes=80/classes=1/g' cfg/yolo-obj.cfg
!sed -i 's/filters=255/filters=18/g' cfg/yolo-obj.cfg
!sed -i 's/width=416/width=608/g' cfg/yolo-obj.cfg
!sed -i 's/height=416/height=608/g' cfg/yolo-obj.cfg

In [ ]:
%pycat cfg/yolo-obj.cfg

2.Create file obj.names in the directory `build\darknet\x64\data\` with objects names - each in new line



In [ ]:
all_classes = """fire
"""

file = """text_file = open("build/darknet/x64/data/obj.names", "w");text_file.write(all_classes);text_file.close()""" 

exec(file)
%pycat build/darknet/x64/data/obj.names

3.Create file obj.data in the directory `build\darknet\x64\data\` containing (where classes = number of objects):

In [ ]:
obj_data = """classes= 1
train  = build/darknet/x64/data/train.txt
valid  = build/darknet/x64/data/valid.txt
names = build/darknet/x64/data/obj.names
backup = build/darknet/x64/backup/
"""

file = """text_file = open("build/darknet/x64/data/obj.data", "w");text_file.write(obj_data);text_file.close()""" 

exec(file)
%pycat build/darknet/x64/data/obj.data

4.Put image-files (.jpg) of your objects in the directory build/darknet/x64/data/obj/


In [ ]:
%cp -r "/content/drive/MyDrive/MPVI/Zavrsni/fire_detection/obj/." build/darknet/x64/data/obj/

In [ ]:
ls -1 "/content/drive/MyDrive/MPVI/Zavrsni/fire_detection/obj/" | wc -l

1013


In [ ]:
%ls -1 build/darknet/x64/data/obj/*.jpg | wc -l
%ls -1 build/darknet/x64/data/obj/*.txt | wc -l

505
506


5.You should label each object on images from your dataset. Use this visual GUI-software for marking bounded boxes of objects and generating annotation files for Yolo v2 & v3: 

---
**LabelImg**
  
LabelImg is a graphical image annotation tool.: 
https://github.com/tzutalin/labelImg


---

**Yolo_mark**

Windows & Linux GUI for marking bounded boxes of objects in images for training Yolo v3 and v2

https://github.com/AlexeyAB/Yolo_mark

It will create `.txt`-file for each `.jpg`-image-file - in the same directory and with the same name, but with `.txt`-extension, and put to file: object number and object coordinates on this image, for each object in new line: `<object-class> <x> <y> <width> <height>`

Where:


*   `<object-class>` - integer object number from 0 to (classes-1)
*   `<x_center> <y_center> <width> <height>` - float values relative to width and height of image, it can be equal from (0.0 to 1.0]
*   for example: `<x> = <absolute_x> / <image_width>` or `<height> = <absolute_height> / <image_height>`
*   atention: `<x_center> <y_center>` - are center of rectangle (are not top-left corner)


For example for img1.jpg you will be created img1.txt containing:


```
1 0.716797 0.395833 0.216406 0.147222
0 0.687109 0.379167 0.255469 0.158333
1 0.420312 0.395833 0.140625 0.166667
```



6.Create file train.txt and valid.txt in directory `build\darknet\x64\data\` with filenames of your images, each filename in new line, with path relative to darknet, for example containing:


In [ ]:
import os, fnmatch

file_list = os.listdir('build/darknet/x64/data/obj/')
pattern = "*.txt"
for filename in file_list:
  if fnmatch.fnmatch(filename, pattern):
    a_file = open('build/darknet/x64/data/obj/'+filename, "r")
    list_of_lines = a_file.readlines()
    for i in range(len(list_of_lines)):
      list_of_lines[i] = '0'+list_of_lines[i][2:]
      #print(list_of_lines[i])
    a_file.close()
    
    a_file = open('build/darknet/x64/data/obj/'+filename, "w")
    a_file.writelines(list_of_lines)
    a_file.close()
    print("done "+filename)

done image_300.txt
done image_320.txt
done image_376.txt
done image_194.txt
done image_281.txt
done image_240.txt
done image_53.txt
done image_119.txt
done image_85.txt
done image_99.txt
done image_178.txt
done image_302.txt
done image_380.txt
done 75.txt
done image_259.txt
done image_179.txt
done image_225.txt
done 34.txt
done image_321.txt
done 79 (1).txt
done image_407.txt
done image_309.txt
done image_34.txt
done image_46.txt
done image_264.txt
done image_18.txt
done 10.txt
done image_23.txt
done image_48.txt
done image_239.txt
done image_269.txt
done image_24.txt
done image_26.txt
done 14.txt
done image_84.txt
done image_243.txt
done image_224.txt
done image_126.txt
done image_304.txt
done image_39.txt
done image_394.txt
done image_335.txt
done image_183.txt
done image_152.txt
done image_187.txt
done image_307.txt
done image_297.txt
done image_133.txt
done image_156.txt
done image_196.txt
done image_25.txt
done image_411.txt
done image_108.txt
done image_127.txt
done image_330.txt

In [ ]:
import os, fnmatch
import numpy as np

train_file = open("build/darknet/x64/data/train.txt", "w")
valid_file = open("build/darknet/x64/data/valid.txt", "w")
listOfFiles = os.listdir('build/darknet/x64/data/obj/')  
pattern = "*.jpg"  
for f_name in listOfFiles:  
  if fnmatch.fnmatch(f_name, pattern):
    if np.random.rand(1) < 0.8:
      train_file.write("build/darknet/x64/data/obj/"+f_name+"\n")
    else:
      valid_file.write("build/darknet/x64/data/obj/"+f_name+"\n")  
      
train_file.close()
valid_file.close()

In [ ]:
#Count number of files 
!wc -l build/darknet/x64/data/train.txt
!wc -l build/darknet/x64/data/valid.txt

390 build/darknet/x64/data/train.txt
115 build/darknet/x64/data/valid.txt


In [ ]:
%pycat build/darknet/x64/data/valid.txt

7.Download pre-trained weights for the convolutional layers (154 MB): https://pjreddie.com/media/files/darknet53.conv.74 and put to the directory `build\darknet\x64`

In [ ]:
!wget -P build/darknet/x64/ https://pjreddie.com/media/files/darknet53.conv.74

--2021-06-09 21:10:51--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘build/darknet/x64/darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  34.2MB/s    in 4.9s    

2021-06-09 21:10:57 (31.8 MB/s) - ‘build/darknet/x64/darknet53.conv.74’ saved [162482580/162482580]



8.training by using the command line:


```
./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/darknet53.conv.74 -dont_show
```

8.1. For training with mAP (mean average precisions) calculation for each 4 Epochs (set valid=valid.txt or train.txt in obj.data file) and run: 



```
./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/darknet53.conv.74 -dont_show -map
```



In [ ]:
import os
os.chdir('drive/MyDrive/MPVI/Zavrsni/fire_detection/WORKING_DIR/darknet/')
!chmod a+x darknet

!./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/darknet53.conv.74 -dont_show

9.After training is complete - get result yolo-obj_final.weights from path `build\darknet\x64\backup\`


*   After each 100 iterations you can stop and later start training from this point. For example, after 2000 iterations you can stop training, and later just copy yolo-obj_2000.weights from `build\darknet\x64\backup\` to` build\darknet\x64\ `and start training using: 


```
./darknet detector train data/obj.data yolo-obj.cfg yolo-obj_2000.weights
```





In [ ]:
!./darknet detector train build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-obj_last2100it.weights -dont_show

Custom object detection:
---
Example of Guns detection:

**Image : **

```
./darknet detector test build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/guns_1000it.weights -thresh 0.20 data/ak47.jpg
```


**Video : **

```
./darknet detector demo build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/guns_1000it.weights -thresh 0.20 -dont_show Guns.mp4 -out_filename Guns_output.mp4
```



In [ ]:
!./darknet detector test build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-obj_last2200it.weights -thresh 0.20 data/frame1.jpg


In [ ]:
!./darknet detector demo build/darknet/x64/data/obj.data cfg/yolo-obj.cfg build/darknet/x64/backup/yolo-obj_last2200it.weights -thresh 0.20 -dont_show test3.mp4 -out_filename test3_output.mp4

In [ ]:
ls -lh *.mp4

-rw------- 1 root root 2.3M Jun 11 11:14 fire20%.mp4
-rw------- 1 root root 1.3M Jun  9 22:36 fire.mp4
-rw------- 1 root root 4.2M Jul  1 19:56 fire_output.mp4


**Playing videos on google colab**

In [ ]:
import io
import base64
from IPython.display import HTML
video = io.open('fire_output.mp4', 'r+b').read()
encoded = base64.b64encode(video)
HTML(data='''<video alt="test" controls><source src="data:video/mp4;base64,{0}" type="video/mp4" /></video>'''.format(encoded.decode('ascii')))

Extra Tutorial
---

Copying data from google colab to google drive

In [ ]:
%cp -r build/darknet/x64/yolo-obj_1500up_05avgLoss.weights "/content/drive/My Drive/GColab/"
%cp -r Guns_output.mp4 "/content/drive/My Drive/GColab/"

Downloading the data from the colab

In [ ]:
from google.colab import files
files.download('build/darknet/x64/yolo-obj_1500up_05avgLoss.weights')

Copying data from google drive to google colab

In [ ]:
%cp -r "/content/drive/My Drive/GColab/yolo-obj_1500up_05avgLoss.weights" build/darknet/x64/
%cp -r "/content/drive/My Drive/GColab/Guns.mp4" .

Youtube Link: 
---
https://youtu.be/Uzty5hRWSQs